In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score

from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [4]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [5]:
mass = pd.read_pickle('../data/03_dummy_df.pickle')

In [6]:
mass.head()

,id,country,ccode,region,protestnumber,protesterviolence,location,protesteridentity,sources,notes,...,shootings,labor_wage_dispute,land_farm_issue,police_brutality,political_behavior_process,price increases_tax_policy,removal_of_politician,social_restrictions,start_date,end_date
0,201990001,Canada,20,North America,1,0.0,national,unspecified,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,...,0,1,0,0,1,0,0,0,1990-01-15,1990-01-15
1,201990002,Canada,20,North America,2,0.0,"Montreal, Quebec",unspecified,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,...,0,0,0,0,1,0,0,0,1990-06-25,1990-06-25
2,201990003,Canada,20,North America,3,0.0,"Montreal, Quebec",separatist parti quebecois,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai...",...,0,0,0,0,1,0,0,0,1990-07-01,1990-07-01
3,201990004,Canada,20,North America,4,1.0,"Montreal, Quebec",mohawk indians,1. indians gather as siege intensifies; armed ...,canada s federal government has agreed to acqu...,...,0,0,1,0,0,0,0,0,1990-07-12,1990-09-06
4,201990005,Canada,20,North America,5,1.0,"Montreal, Quebec",local residents,1. dozens hurt in mohawk blockade protest the ...,protests were directed against the state due t...,...,0,0,0,0,1,0,0,0,1990-08-14,1990-08-15


In [7]:
list(mass.columns)

['id',
 'country',
 'ccode',
 'region',
 'protestnumber',
 'protesterviolence',
 'location',
 'protesteridentity',
 'sources',
 'notes',
 'protester_id_type',
 'partipants_number',
 'protest_size_category',
 'pop_male',
 'pop_female',
 'pop_total',
 'pop_density',
 'prosperity_2020',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'coun

In [8]:
mass.shape

(15225, 207)

In [9]:
mass[[ 'protester_id_type']]

,protester_id_type
0,workers_unions
1,students_youth
2,political_group
3,ethnic_group
4,locals_residents
...,...
15220,locals_residents
15221,students_youth
15222,political_group
15223,political_group


In [10]:
mass[['protesterviolence']]

,protesterviolence
0,0.0
1,0.0
2,0.0
3,1.0
4,1.0
...,...
15220,1.0
15221,1.0
15222,0.0
15223,1.0


Dummify `protest_size_category` and `protester_id_type`  columns. 

In [11]:
mass = pd.get_dummies(mass, columns=['protest_size_category', 'protester_id_type'], dummy_na=True)

In [12]:
# list(mass.columns)

Dropping `nan` columns created from dummifying `protest_size_category` and `protester_id_type`  columns. 

In [13]:
mass.drop(columns=['protest_size_category_nan', 'protester_id_type_nan'], inplace=True)

In [14]:
features = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])

In [15]:
# mass['start_date']

Below, attempting a Logistic Regression for each state response, starting with `arrests`.

In [16]:
X = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])
y = mass['arrests']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

ss = StandardScaler()

X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [17]:
X_train_sc

array([[ 1.57789781,  0.55705004, -0.59783405, ..., -0.04625163,
        -0.15395279, -0.55509474],
       [-0.74828518,  1.71833142, -0.59783405, ..., -0.04625163,
        -0.15395279,  1.80149429],
       [ 0.0327984 , -0.27243667,  1.672705  , ..., -0.04625163,
        -0.15395279, -0.55509474],
       ...,
       [ 1.45410895, -0.189488  ,  1.672705  , ..., -0.04625163,
        -0.15395279, -0.55509474],
       [ 0.36572147,  0.88884472, -0.59783405, ..., -0.04625163,
        -0.15395279, -0.55509474],
       [-1.13243169, -0.189488  , -0.59783405, ..., -0.04625163,
        -0.15395279,  1.80149429]])

In [18]:
ss = StandardScaler()

X_sc = ss.fit_transform(X)

In [19]:
lr = LogisticRegression(solver='lbfgs', max_iter=1000)

lr.fit(X_train_sc, y_train)

LogisticRegression(max_iter=1000)

In [20]:
lr.score(X_train_sc, y_train)

0.8637931034482759

In [21]:
lr.score(X_test_sc, y_test)

0.851888341543514

In [22]:
lr.predict(X_test_sc)

array([0, 0, 0, ..., 0, 0, 1])

In [23]:
predictions = lr.predict(X_test_sc)
predictions

array([0, 0, 0, ..., 0, 0, 1])

In [24]:
probabilities = lr.predict_proba(X_test_sc) 
probabilities

array([[0.59839355, 0.40160645],
       [0.9015369 , 0.0984631 ],
       [0.88858778, 0.11141222],
       ...,
       [0.86149826, 0.13850174],
       [0.68240283, 0.31759717],
       [0.3996736 , 0.6003264 ]])

In [25]:
probs_all_arrests = lr.predict_proba(X_sc)[:,1]

In [26]:
probs_all_arrests

array([0.10456818, 0.19099977, 0.20518939, ..., 0.02467354, 0.07344052,
       0.01363735])

In [27]:
preds_all_arrests = lr.predict(X_sc) 

In [28]:
preds_all_arrests

array([0, 0, 0, ..., 0, 0, 0])

In [29]:
from sklearn.metrics import recall_score

In [30]:
lr

LogisticRegression(max_iter=1000)

In [31]:
mass['arrests_probability'] = probs_all_arrests

mass['arrests_predicted'] = preds_all_arrests

In [32]:
mass[['arrests', 'arrests_predicted','arrests_probability']][:22]

,arrests,arrests_predicted,arrests_probability
0,0,0,0.104568
1,0,0,0.191000
2,0,0,0.205189
3,0,1,0.533197
4,1,0,0.421843
5,0,0,0.200124
6,0,0,0.118751
7,0,0,0.117835
8,1,1,0.521113
9,0,0,0.168790


In [33]:
probabilities[333]

array([0.80489407, 0.19510593])

In [34]:
predictions[333]

0

In [35]:
# X_train['arrests_probability']= lr.predict_proba(X_train_sc)[0][1]

In [36]:
# class StateResponsePredictor:
    
#     def __init__(self):
#         self.state_response_list = state_response_list
        
#     def fit(self, mass):
#         self.ss = StandardScaler()
#         X = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
#                               'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']) 
        

In [135]:
mass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15225 entries, 0 to 15224
Columns: 227 entries, id to arrests_predicted
dtypes: datetime64[ns](2), float64(7), int32(180), int64(12), object(6), uint8(20)
memory usage: 14.6+ MB


In [137]:
mass[['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']].value_counts(normalize=True)

arrests  accomodation  beatings  crowddispersal  ignore  killings  shootings
0        0             0         0               1       0         0            0.524335
                                 1               0       0         0            0.160985
         1             0         0               0       0         0            0.058128
1        0             0         1               0       0         0            0.053596
                                 0               0       0         0            0.041511
                                                                                  ...   
                                 1               1       1         1            0.000066
0        1             0         0               1       0         1            0.000066
1        0             0         0               1       1         1            0.000066
                                                         0         1            0.000066
                       1         

In [134]:
mass[['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']]

,arrests,accomodation,beatings,crowddispersal,ignore,killings,shootings
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0
4,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...
15220,0,0,0,1,0,0,1
15221,0,0,0,1,0,1,1
15222,0,1,0,0,0,0,0
15223,0,0,0,1,0,0,0


In [89]:
# drop_from_X = ['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
#                               'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']
# state_responses = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']

# response_dict = {}

current_columns = mass.drop(cols= ['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])

lr_2 = LogisticRegression(...)

def state_response_predictor(model, df, features, target):
    
    X = features 
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

    ss = StandardScaler()

    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    model.fit(X_train_sc, y_train)
    
    y_preds = model.predict(X_test_sc)
    
    recall_test
    recall_train
    
    Testing se
    
    f1_score, precision, recall = evaluate_model(y_test, y_preds) 
    
    print(f'Training set recall for predictions of {target}: {recall_train)}')
    print(f'Testing set recall for predictions of {target}: {lr.score(X_test_sc)}')
    
    
def add_columns:    
    X_sc = ss.fit_transform(X)
    
    predictions = lr.predict(X_sc) 
    probabilities = lr.predict_proba(X_sc)[:,1]
    
    return predictions, probabilites

In [ ]:
eval_dict = {'arrests':[f]}

In [ ]:
def evaluate_model(y_true, y_preds):
    recall = 
    f1 = 
    return recall, f1 

In [ ]:
mass['arrests_prediction'], mass['probabilites']

In [89]:
    for response in state_responses: 
        
        y = mass[response]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)
        
        X_train_sc = ss.fit_transform(X_train)
        X_test_sc = ss.transform(X_test)
        
        lr = LogisticRegression(solver='lbfgs', max_iter=100)

        lr.fit(X_train_sc, y_train)
        
        response_dict[response] = lr.predict_proba(X_test_sc)[0][1]
        print(response_dict)
        
    top_response = max(response_dict.items(), key=lambda k: k[1])
    
    return top_response 

In [90]:
state_response_predictor(mass, current_cols, 'arrests')

/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'arrests': 0.4016257727256075}
{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955}


/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904}


/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904, 'crowddispersal': 0.3818345169499125}
{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904, 'crowddispersal': 0.3818345169499125, 'ignore': 0.07664528924354543}


/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904, 'crowddispersal': 0.3818345169499125, 'ignore': 0.07664528924354543, 'killings': 0.35658368862007894}
{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904, 'crowddispersal': 0.3818345169499125, 'ignore': 0.07664528924354543, 'killings': 0.35658368862007894, 'shootings': 0.3092331274246054}


/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


('arrests', 0.4016257727256075)

In [ ]:
mass['is_arrests'] = []

In [39]:
mass.groupby('country').count()[mass['3_digit_code'].isnull()]

pandas.core.frame.DataFrame

In [ ]:
mass[mass['country'].notna() & mass['country_alpha_3']].groupby('country')